In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/RETINAL/

/content/drive/MyDrive/RETINAL


In [8]:
!ls

augment.ipynb  models	test	  training	unet-2.ipynb
files	       outputs	test.zip  training.zip	unet.ipynb


In [13]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import HorizontalFlip, VerticalFlip, Rotate, RandomBrightnessContrast

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

def augment_data(images, masks, save_path, index=0, augment:bool=True):
    size = (512, 512)

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the name """
        name = x.split("/")[-1].split(".")[0]

        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0]

        if augment:
            aug = HorizontalFlip(p=0.3)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=0.3)
            augmented = aug(image=x1, mask=y1)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = Rotate(limit=45, p=0.9)
            augmented = aug(image=x2, mask=y2)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            aug = RandomBrightnessContrast(p=0.2)
            augmented = aug(image=x3, mask=y3)
            x4 = augmented["image"]
            y4 = augmented["mask"]

            X = [x4]
            Y = [y4]

        else:
            X = [x]
            Y = [y]

        for i, m in zip(X, Y):
            i = cv2.resize(i, size)
            m = cv2.resize(m, size)

            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.png"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)

In [ ]:
""" Seeding """
np.random.seed(42)

""" Load the data """
data_path = "./"
(train_x, train_y), (test_x, test_y) = load_data(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

""" Create directories to save the augmented data """
create_dir("new_data/train/image/")
create_dir("new_data/train/mask/")
create_dir("new_data/test/image/")
create_dir("new_data/test/mask/")

""" Data augmentation """
for i in range(20):
  augment_data(train_x, train_y, "new_data/train/", index=i, augment=True)

augment_data(test_x, test_y, "new_data/test/", index=0, augment=False)

In [11]:
%ls new_data/test/image

01_test_0.png  05_test_0.png  09_test_0.png  13_test_0.png  17_test_0.png
02_test_0.png  06_test_0.png  10_test_0.png  14_test_0.png  18_test_0.png
03_test_0.png  07_test_0.png  11_test_0.png  15_test_0.png  19_test_0.png
04_test_0.png  08_test_0.png  12_test_0.png  16_test_0.png  20_test_0.png
